In [1]:
import json
from pprint import pprint


In [98]:

import pandas as pd
import numpy as np
from datetime import datetime
from enum import Enum
from abc import ABC, abstractmethod

In [99]:
#a = open('feeddata_10.json')


In [100]:
#'KR is for Korea
#4 for derivatives
#'Then type 1 for future ,2 for call, 3 for put
#'0 no clue
#'Then 1 for maxi or 5 for mini
#'Then 2 letters expiry code, 1 one is year, second month
#'Then 3 digits for the strike
#'Then 1 for isin checksum


In [101]:
data = json.load(open('feeddata_10.json'))

In [102]:
#aPD = pd.DataFrame.from_records(data)
#books = aPD.book.drop_duplicates().str.split('01')
#books = aPD.book.drop_duplicates() #check 1 if maxi 5 if mini
#aTest = books.to_frame

In [103]:
class CONTRACTTYPE(Enum):
    UNSET = 0
    FUTURE = 1
    CALL = 2
    PUT = 3
    
#class Strike(Enum):
#    for i in range(50)+100:
#         str(i+2.5*i) = i-99

In [204]:
# class Books():
    
#     contractType = CONTRACTTYPE.UNSET
#     book = []

    
#     def __init__(self,*args):
#         self.contractType = args[0]
#         if args[1]:
#             self.strike = args[1]
        
#     def setStrike(aStrike):
#         self.strike = aStrike

#     def updateBook(aTickBook):
#         self.book = aTickBook
#         self.updateTime = ''
#         self.timeToExpiry = ''
        
#     def getVolume(level, isBid):
#         return 0
        
#     def getBestBid():
#         return 0
    
#     def getBestAsk():
#         return 0

In [206]:
class aSimpleBook():
    
    aStrikeBook = {}
    
    def getBestBid_Price(self,aStrike):
        return aStrikeBook["aStrike"]['bid_price_0']
    
    def getBestAsk_Price(self,aStrike):
        return aStrikeBook["aStrike"]['ask_price_0']
    
    def getBestBid_Volume(self,aStrike):
        return aStrikeBook["aStrike"]['bid_volume_0']
    
    def getBestAsk_Volume(self,aStrike):
        return aStrikeBook["aStrike"]['ask_volume_0']
    
    def updateBook(self,aNewTick):
        self.aStrikeBook["aStrike"] = aNewTick
        


In [192]:
class Traders(ABC):


    @abstractmethod
    def getPnL(self):
        pass
    @abstractmethod
    def getPosition(self):
        pass
    @abstractmethod
    def update(self):
        pass

In [193]:
class TraderTest(Traders):
    Position = {}
    LastFutureValue = 0
    cashPnL = 0
    PnL = 0
    theStrategies = []
    theBook = aSimpleBook
    
    def __init__(self, aBunchOfStrategies):
        theStrategies = aBunchOfStrategies
    
    def getPnL(self):
        return self.PnL
    
    def getPosition(self, strike): #options
        return self.Position['Strike']
    
    def getPosition(self):  #futures
        return self.Position['000']
    
    def updatePnL(self, aVolumeChange,aStrike):
        
        self.PnL = 0
        
        if(aVolumeChange > 0):  
            cashPnL = cashPnL - aVolumeChange*self.theBook[aStrike]['ask_price_0']  #only crossing the spread here, assuming always enough volume
        elif(aVolumeChange < 0):
            cashPnL = cashPnL - aVolumeChange*self.theBook[aStrike]['bid_price_0']
            
        for key in Position.keys:
            self.PnL = self.PnL + self.Position['key']*(self.theBook[key]['ask_price_0']+self.theBook[key]['bid_price_0'])/2
        
        self.PnL = self.PnL + cashPnL
        
        return self.PnL
    
    def update(self,aTick):
        
        self.theBook.updateBook(aTick["rawStrikeBook"])
            
        for strat in theStrategies:
            volumeChange = strat.getAction(aTick, theBook)
            self.Position[aTick["Strike"]] = self.Position[aTick["Strike"]] + volumeChange
        
        myPnL = self.updatePnL(volumeChange,aTick["Strike"])
        return myPnL
        

In [194]:
class Strategy(ABC):
    
    @abstractmethod
    def getAction(self,aTick):
        return 0

In [195]:
class StrategyTest():
    
    def getAction(aTick, LastFutureValue, aBook):
        
        myQty = 0 
        
        if(aTick["contract"] == CONTRACTTYPE.CALL):
            if aTick["midPoint"] > LastFutureValue - float(aTick["Strike"])*getDiscount(aTick) + putPrice:
                myQty = -1 
                
                
        return myQty
        
        
    
    
    

In [196]:
def isCorrectISINCHECK(aISIN):
    #TODO
    return True

In [197]:
def getDiscount(aTick):
    
    rate = 0.025
    
    return np.exp( -rate * (aTick["timeToExpiry"].days + aTick["timeToExpiry"].seconds / (60*60*24))/365)
    
    

In [198]:
import calendar
import sys
from dateutil import tz

def getExpiryDate(anExpiryCode):
    #hardcoded version for now. 
    myString_date = ''
    
    myString_year = ''
    myString_month = ''
    myString_day = ''
    
    #get year code
    if anExpiryCode[0] == 'L':
        myString_year = "2016"
    elif anExpiryCode[0] == 'M':
        myString_year = "2017"
    elif anExpiryCode[0] == 'N':
        myString_year = "2018"
    elif anExpiryCode[0] == 'P': #It looks strange, but that's KRX 
        myString_year = "2019"
    else : 
        sys.stderr.write("fatal error\n expiry year code unknown \n")
    
    #get month code
    myPotentialMonths = ['','1','2','3','4','5','6','7','8','9','A','B','C']

    if anExpiryCode[1] in myPotentialMonths:
        myString_month =  str(myPotentialMonths.index(anExpiryCode[1]))
    else : 
        sys.stderr.write("fatal error\n expiry month code unknown \n")    
    
    if myString_month == '':
        sys.stderr.write("fatal error\n expiry month code unknown \n")   
    #get day
    #get third Wednesday of every month, we will have to make bank holidays exceptions
    myString_day = str(calendar.Calendar(2).monthdatescalendar(int(myString_year), \
                                                               int(myString_month))[3][0].day)
    
                       
    myString_date = myString_year+'-'+myString_month+'-'+myString_day+' 16:00:00'

    timeZone = tz.gettz('Asia/Seoul')
            
    #print("Don't forget bank holidays exceptions")
    return datetime.strptime(myString_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timeZone).astimezone(timeZone)


In [199]:

def utcToRightTMZ(aUTCTMZ):
    
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('Asia/Seoul')
    
    aUTCTMZ = aUTCTMZ.replace(tzinfo=from_zone)
    localTimeStamp = aUTCTMZ.astimezone(to_zone)
    
    return localTimeStamp

In [200]:
def getBookContract(aStringNumberContract, aTypeOption):
    aContract = ''
    if aStringNumberContract == "000" and aTypeOption == "1":
        aContract = CONTRACTTYPE.FUTURE #replace with enums
    elif aTypeOption == "2":
        aContract = CONTRACTTYPE.CALL #replace with enums
    elif aTypeOption == "2":
        aContract = CONTRACTTYPE.CALL #replace with enums
    else:
        print("error, Contract Type" + str(aStringNumberContract) + " , " + str(aTypeOption) + " not recognized")
    
    return aContract

In [201]:
class simulationCrunsher():
    
    def __init__(self, strategies_):
        self.theTrader = TraderTest(strategies_)
    
    def run(self, data):
        counter = 0
        myFirstLastDone = True

        for tick in data:
            counter = counter + 1 
            myTick = dict()

            myBook = tick["book"]
            if not isCorrectISINCHECK(myBook):
                sys.stderr.write("fatal error\n ISIN CHECK IS BREAKING \n")   

            if myBook[5] == '1' and  tick['type']=='tick':   #let's forget about mini futures
                try:
                    #print(myBook)
                    myTick["contract"] = getBookContract(myBook[8:11],myBook[3])

                    timeStamp = utcToRightTMZ(datetime.utcfromtimestamp(tick["created"]/1000000))
                    #print(timeStamp)
                    myTick["created"] = timeStamp
                    myTick["received"] = utcToRightTMZ(datetime.utcfromtimestamp(tick["received"]/1000000))

                    myTempStrike = data[1]['book'][8:11]

                    if(myTick["contract"]  == CONTRACTTYPE.CALL or myTick["contract"]  == CONTRACTTYPE.PUT):
                        myTick["Strike"] = myBook[8:11]
                    #print(myBook[6:8])
                    myTick["expiry"] = getExpiryDate(myBook[6:8])

                    myTick["spread"] = (tick['ask_price_0']-tick['bid_price_0'])/2
                    myTick["midPoint"] =(tick['ask_price_0']+tick['bid_price_0'])/2

                    myTick["timeToExpiry"] = myTick["expiry"] - myTick["received"]
                    
                    myTick["rawStrikeBook"] = tick
                    
                    pnL = self.theTrader.update(myTick)
                    
                except Exception as e: 
                    print(e)
                    print(tick)
                    break

            elif myBook[5] == '1' and tick['type']=='lastdone':
                if myFirstLastDone: 
                    print("@TODO : Handle Last Done")
                myFirstLastDone = False
            elif myBook[5] == '1':
                print(tick)

            if counter > 2000 :
                break

example:

In [202]:
tick

{'ask_price_0': 3.91,
 'ask_price_1': 3.92,
 'ask_price_2': 3.93,
 'ask_price_3': 3.94,
 'ask_price_4': 3.95,
 'ask_volume_0': 3,
 'ask_volume_1': 26,
 'ask_volume_2': 8,
 'ask_volume_3': 17,
 'ask_volume_4': 44,
 'bid_price_0': 3.9,
 'bid_price_1': 3.89,
 'bid_price_2': 3.88,
 'bid_price_3': 3.87,
 'bid_price_4': 3.86,
 'bid_volume_0': 1,
 'bid_volume_1': 8,
 'bid_volume_2': 9,
 'bid_volume_3': 10,
 'bid_volume_4': 7,
 'book': 'KR4201MB3355',
 'created': 1509498004173059,
 'received': 1509498004173475,
 'type': 'tick'}

In [207]:
class runners():
    theStrats = [StrategyTest]
    
    theSimulator = simulationCrunsher(theStrats)
    
    theSimulator.run(data)
    

updateBook() missing 1 required positional argument: 'aNewTick'
{'received': 1509498000001973, 'created': 1509498000001577, 'book': 'KR4201MC3404', 'type': 'tick', 'ask_price_0': 4.15, 'ask_volume_0': 1, 'ask_price_1': 4.16, 'ask_volume_1': 6, 'ask_price_2': 4.17, 'ask_volume_2': 1, 'ask_price_3': 4.18, 'ask_volume_3': 1, 'ask_price_4': 4.2, 'ask_volume_4': 1, 'bid_price_0': 4.07, 'bid_volume_0': 1, 'bid_price_1': 4.06, 'bid_volume_1': 2, 'bid_price_2': 4.05, 'bid_volume_2': 7, 'bid_price_3': 4.04, 'bid_volume_3': 5, 'bid_price_4': 4.03, 'bid_volume_4': 6}


SIMPLE EACD MODEL : 

In [42]:
# if 'xs' in globals() or 'psis' in globals():
#     sys.stderr.write('xs or/and psis variables already exist')
#else:

#variables of Interest:
psis = []
xs = []

#parameters:
alpha0 = 0
alphas = []
betas = []


def residualTotalEACD(psis_, xs_, alpha0_, alphas_, betas_):
    
    alphaSum = 0
    betaSum = 0
    errs = []
    
    p = len(alphas)
    q = len(betas)
    
    for idx,alpha in enumerate(alphas):        
        alphaSum = alphaSum + alphas[idx]*xs[idx] #doesn't matter if sum in reverse order, just means that alphas are in reverse order too
    for idx,beta in enumerate(betas):
        betaSum = betaSum + betas[idx]*psis[idx]
        
    
    initialized = False
    first = True
    for idx, tick in enumerate(data):
        
        if initialized and not first :
            xs.append(datetime.utcfromtimestamp(tick["created"]/1000000) - myPastTime)
            expectedTime = alpha0 + alphaSum +  betaSum
            
            errs[idx] = (expectedTime - xs[idx])^2
            
            pastExpectedTime = 
            if counter > 2000 :
                break
        elif first:
            first = False
        else:
            xs.append(datetime.utcfromtimestamp(tick["created"]/1000000) - myPastTime)
            if len(xs) == p:
                initialized = True
                
        myPastTime = datetime.utcfromtimestamp(tick["created"]/1000000)
        
        #maintain right array size
        if len(xs)>p:
            xs.pop(0)
        if len(psis)>q:
            psis.pop(0)    
    return 

In [33]:
roh = []

In [38]:
roh.append(1)
roh.append(2)
roh.append(3)
roh.append(4)

In [40]:
roh.pop(2)

4

In [41]:
roh

[1, 2, 1, 2, 3, 4]